# General Flow for Training/Fitting Models

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

from data_utils import PCA, RandomForestClassifier, StandardScaler
from data_utils import classification_error, display_confusion_matrix
from data_utils import object_from_json_url

### 3 Stages
- Data Prep: Encoding, Scaling, PCA, sometimes Splitting into train/test datasets
- Modeling: `fit()` classifier
- Evaluation: predict and measure error

#### Data Prep:
Do we need to split our data, or is it already split into train/test sets?

If it's already split we prepare the Encoding, Scaling, PCA objects using the `train` data (usually with the `fit_transform()` function), and then we use those same objects to encode, scale, PCA the `test` data (usually with the `transform()` function).

If the data is not split into two datasets, we could first split it and repeat the steps above, or, although it might add a bit of bias to the models, we could perform Encoding, Scaling, PCA with `fit_transform()` on the entire dataset and then only split the already encoded, scaled, PCA'ed data. This biases the encoder, scaler, PCA models, and in turn, the model, but is a bit easier to perform.

#### Modeling
Once we have `train` and `test` datasets that has been encoded, scaled, PCA'ed, we can use the `train` dataset to fit a supervised model (classifier, regression, etc).

Here we will usually call a `fit()` function with the training dataset's features and, separately, its labels or outcome variable values. Something like `fit(features, labels)`.

#### Evaluation
We have a model we trained/fitted with the `train` dataset. Now we can measure how well it actually performs once it's used without the correct labels.

Here we usually call `predict()` with a dataset's features to get label or regression predictions.

We want to call `predict()` for both the `train` and `test` dataset, and then measure how close those predictions are to the actual labels and values that we have in our dataset.

Eavluating with the `train` dataset will tell us if the model is capable of learning anything about the data. Evaluating with the `train` dataset will tell us if the model is capable of learning patterns and trends beyond the data that is fed to it.

It's common for the model to perform better with the `train` data since it was trained using that data and labels, but the `test` dataset error is what's more important because it will tell us what kind of error to expect from data that the model hasn't seen.

### Example

Classifying penguins based on measurements.

Let's load a dataset and look.

In [ ]:
PENGUIN_URL = "https://raw.githubusercontent.com/DM-GY-9103-2024F-H/9103-utils/refs/heads/main/datasets/json/penguins.json"
penguin_data = object_from_json_url(PENGUIN_URL)

display(penguin_data)

It doesn't have separate train and test data, so we can either 

#### Pre-process and then split:

<img src="./imgs/datasplit-00.jpg" width="720px"/>

OR
#### Split and then process:

<img src="./imgs/datasplit-01.jpg" width="720px"/>

In [ ]:
# TODO: Put in DataFrames
penguin_df = pd.DataFrame(penguin_data)
display(penguin_df)
display(penguin_df.shape)

# TODO: Split + Process
species = penguin_df["species"].unique().tolist()

# Encode
def S2L(S):
  return species.index(S)

penguin_df["label"] = penguin_df["species"].apply(S2L)
display(penguin_df)
display(penguin_df.shape)

### Split the Data

Using `train_test_split()`

In [ ]:
# TODO: split with train_test_split()
penguin_train, penguin_test = train_test_split(penguin_df, train_size=0.8)

In [ ]:
penguin_train.shape, penguin_test.shape

In [ ]:
penguin_scaler = StandardScaler()
train_scaled_df = penguin_scaler.fit_transform(penguin_train.drop(columns=["species", "label"]))

penguin_pca = PCA(n_components=3)
train_pca_df = penguin_pca.fit_transform(train_scaled_df)

train_pca_df["label"] = penguin_train["label"].values
display(train_pca_df)
train_pca_df.shape

In [ ]:
# Transform Test data
test_scaled_df = penguin_scaler.transform(penguin_test.drop(columns=["species", "label"]))
test_pca_df = penguin_pca.transform(test_scaled_df)

test_pca_df["label"] = penguin_test["label"].values
display(test_pca_df)
test_pca_df.shape

### Check Sizes

In [ ]:
# TODO: Check sizes
train_pca_df.shape, test_pca_df.shape

### Model/Fit

We can train our model now. We're going to use a `RandomForestClassifier` and `fit()` it with the training data.

In [ ]:
# TODO: fit RandomForestClassifier
penguin_model = RandomForestClassifier()
penguin_model.fit(train_pca_df.drop(columns=["label"]), train_pca_df["label"])

### Evaluate

We can now run predictions for both `train` and `test` data and measure error.

In [ ]:
# TODO: predict() train and test
train_pred = penguin_model.predict(train_pca_df.drop(columns=["label"]))
test_pred = penguin_model.predict(test_pca_df.drop(columns=["label"]))

Before measuring the error we can check to see if these predictions have the right shapes and values.

In [ ]:
# check sizes
train_pred.shape, test_pred.shape

### Measure Error

In [ ]:
# TODO: measure classification error with classification_error()
display(classification_error(train_pca_df["label"], train_pred))
display(classification_error(test_pca_df["label"], test_pred))

### Look at Confusion (Matrix)

`display_confusion_matrix(labels, predictions, display_labels=unique_labels)`

In [ ]:
# TODO: look at confusion matrices
display_confusion_matrix(train_pca_df["label"], train_pred, display_labels=penguin_df["species"].unique().tolist())
display_confusion_matrix(test_pca_df["label"], test_pred, display_labels=penguin_df["species"].unique().tolist())